In [ ]:
import pandas as pd

activities = pd.read_csv('viral_activities.csv')

In [ ]:
import sqlite3

conn = sqlite3.connect('/tmp/chembl_35/chembl_35_sqlite/chembl_35.db')
cursor = conn.cursor()

# lets get the higher ranks

In [ ]:
taxid2rank = {}

with open("/tmp/nodes.dmp") as f:
    for line in f:
        parts = [p.strip() for p in line.split("|")]
        taxid2rank[int(parts[0])] = parts[2]

In [ ]:
set(taxid2rank.values())

In [ ]:
from pathlib import Path
import json

json.dump(taxid2rank, Path('taxid2rank.json').open('w'))

In [ ]:
from boilerplate import TaxIdMapper

taxid_mapper = TaxIdMapper()

In [ ]:
taxid_mapper.taxid_to_parent

In [ ]:
from tqdm import tqdm
tqdm.pandas()

def get_rank_id(tax_id, rank_name='genus'):
    while True:
        if tax_id in (-1, 0, 1, 2, 10239):
            tax_id = -1
            break
        tax_id = taxid_mapper.taxid_to_parent.get(tax_id, -1)
        if taxid2rank.get(tax_id, '') == rank_name:
            return tax_id
    return -1

def get_genus_id(tax_id):
    return get_rank_id(tax_id, rank_name='genus')

def get_family_id(tax_id):
    return get_rank_id(tax_id, rank_name='family')

def get_class_id(tax_id):
    return get_rank_id(tax_id, rank_name='class')

def get_phylum_id(tax_id):
    return get_rank_id(tax_id, rank_name='phylum')

def get_realm_id(tax_id):
    return get_rank_id(tax_id, rank_name='realm')


activities['genus_id'] = activities.assay_tax_id.progress_apply(get_genus_id)
activities['family_id'] = activities.assay_tax_id.progress_apply(get_family_id)
activities['class_id'] = activities.assay_tax_id.progress_apply(get_class_id)
activities['phylum_id'] = activities.assay_tax_id.progress_apply(get_phylum_id)
activities['realm_id'] = activities.assay_tax_id.progress_apply(get_realm_id)

In [ ]:
activities.to_csv('viral_activities.csv')

In [ ]:
from typing import Dict

taxid_to_name: Dict[int, str] = globals().get('taxid_to_name', {})

wanted = []
for col in ('genus_id', 'family_id', 'class_id', 'phylum_id', 'realm_id'):
    wanted.extend( activities[col].unique() )

with open("/tmp/names.dmp") as f:
    for line in f:
        parts = [p.strip() for p in line.split("|")]
        taxid, name_txt, name_class = parts[0], parts[1], parts[3]
        if int(taxid) in wanted and name_class == "scientific name":
            taxid_to_name[int(taxid)] = name_txt

In [ ]:
len(taxid_to_name)

In [ ]:
summary = \
activities.drop_duplicates(['assay_tax_id','molregno'])\
          .groupby(['standard_type', 'realm_id'])\
          .size().unstack(fill_value=0)\
          .drop(columns=[-1])\
          .loc[activities.standard_type.value_counts().head(10).index]\
          .rename(columns=taxid_to_name)
summary.style.format("{:,}")

In [ ]:
summary = \
activities.drop_duplicates(['assay_tax_id','molregno'])\
          .groupby(['standard_type', 'phylum_id'])\
          .size().unstack(fill_value=0)\
          .drop(columns=[-1])\
          .loc[activities.standard_type.value_counts().head(6).index]\
          .rename(columns=taxid_to_name)
summary.style.format("{:,}")

In [ ]:
summary = \
activities.drop_duplicates(['assay_tax_id','molregno'])\
          .groupby(['standard_type', 'family_id'])\
          .size().unstack(fill_value=0)\
          .drop(columns=[-1])\
          .loc[activities.standard_type.value_counts().head(6).index]\
          .rename(columns=taxid_to_name)
summary.style.format("{:,}")

In [ ]:
from boilerplate import molar_scale
import numpy as np

activities['unit_scale'] = activities.standard_units.fillna('').apply(molar_scale)
activities['pk'] = - (activities.unit_scale * activities.standard_value).apply(np.log10)

In [ ]:
placeholders = ', '.join(activities.molregno.astype(str).apply("'{}'".format).unique())

query = f"SELECT * FROM COMPOUND_STRUCTURES WHERE molregno IN ({placeholders})"
molregno = pd.read_sql_query(query, conn)

In [ ]:
from rdkit import Chem

molregno['mol'] = molregno.molfile.apply(Chem.MolFromMolBlock)

In [ ]:
molregno['HAC'] = molregno.mol.apply(Chem.Mol.GetNumHeavyAtoms)

In [ ]:
from rdkit.Chem import Draw

for genus_id, _ in activities.drop_duplicates(['genus_id','molregno']).genus_id.value_counts().items():
    if genus_id == -1:
        continue
    genus_name = taxid_to_name[genus_id]
    print(genus_name)
    top50 = activities.loc[activities.genus_id == genus_id]\
                      .sort_values('pk', ascending=False)\
                      .drop_duplicates(['molregno'])\
                      .head(50)
    if len(top50) == 0:
        continue
    top50['mol'] = top50.molregno.map(molregno.set_index('molregno').mol.to_dict())
    top50 = top50.loc[~top50.mol.isna()]
    legends: pd.Series = (top50.standard_type.astype(str).str.replace(' ','_') +'='+top50.standard_value.astype(str)+top50.standard_units.astype(str))
    png = Draw.MolsToGridImage(top50.mol.to_list(),
                         molsPerRow=5,
                         legends=legends.to_list())
    png.save(f'{genus_name}.png')

In [ ]:
# for genus_id, _ in activities.drop_duplicates(['genus_id','molregno']).genus_id.value_counts().items():
#     if genus_id == -1:
#         continue
#     genus_name = taxid_to_name[genus_id]
#     print(f'#### {genus_name}\n\n![{genus_name}](images/{genus_name}.png)\n')

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform

print('Generating fingerprints...')

def calculate_morgan_fingerprints(mol_series, radius=2, n_bits=1024):
    fingerprints = []
    generator = rdFingerprintGenerator.GetMorganGenerator( radius=radius, fpSize=n_bits)
    for mol in mol_series:
        fp = generator.GetFingerprintAsNumPy(mol)
        fingerprints.append(fp)
    return np.array(fingerprints)

molregno['mol'] = molregno.mol.fillna(Chem.Mol())
fingerprints = calculate_morgan_fingerprints(molregno.mol)

print('Compute Tanimoto distances...')
def tanimoto_distance(x, y):
    return 1 - np.sum(x & y) / np.sum(x | y)

tanimoto_distances = pdist(fingerprints, metric=tanimoto_distance)
distance_matrix = squareform(tanimoto_distances)

In [ ]:
print('Perform t-SNE...')
tsne = TSNE(metric='precomputed', init='random')
tsne_results = tsne.fit_transform(distance_matrix)

molregno['tSNE1'] = tsne_results[:, 0]
molregno['tSNE2'] = tsne_results[:, 1]

In [ ]:
def get_genera(molregno):
    genera = activities.genus_id.loc[activities.molregno == molregno].unique()
    named_genera = [taxid_to_name[g] for g in genera if g != -1]
    return '+'.join(sorted(named_genera))

molregno['in_genera'] = molregno.molregno.apply(get_genera)

In [ ]:
def get_families(molregno):
    families = activities.family_id.loc[activities.molregno == molregno].unique()
    named_families = [taxid_to_name[g] for g in families if g != -1]
    return '+'.join(sorted(named_families))

molregno['in_families'] = molregno.molregno.apply(get_families)

In [ ]:
import plotly.express as px

# aggregating values
molregno['in_families_trimmed'] =  molregno.in_families.apply(lambda v: v if v.count('+') == 0 else 'multiple families')
family_counts = molregno['in_families_trimmed'].value_counts()
rename_map = {name: name if counts >= 100 else 'less studied families' for name, counts in family_counts.items()}
molregno['in_families_trimmed'] =  molregno['in_families_trimmed'].map(rename_map)

label_order = family_counts.index.to_list()
label_order.pop(label_order.index('multiple families'))
label_order.append('multiple families')
label_order.append('less studied families')

px.scatter(molregno.loc[(molregno.HAC < 60) & (molregno.HAC >= 20)], 'tSNE1', 'tSNE2', color='in_families_trimmed',
          template='plotly_white',
           category_orders={'in_families_trimmed': label_order},
          title='Diversity of compounds with between 20–60 heavy atoms (tSNE by Tanomoto similarity on 1024bit Morgan fingerprints)',
           labels={'in_families_trimmed': 'Family'},
           height=800,
          )

In [ ]:
import plotly.express as px

# aggregating values
molregno['in_genera_trimmed'] =  molregno.in_genera.apply(lambda v: v if v.count('+') == 0 else 'multiple genera')
genus_counts = molregno['in_genera_trimmed'].value_counts()
rename_map = {name: name if counts >= 100 else 'less studied genera' for name, counts in genus_counts.items()}
molregno['in_genera_trimmed'] =  molregno['in_genera_trimmed'].map(rename_map)

label_order = genus_counts.index.to_list()
label_order.pop(label_order.index('multiple genera'))
label_order.append('multiple genera')
label_order.append('less studied genera')

px.scatter(molregno.loc[(molregno.HAC < 60) & (molregno.HAC >= 20)], 'tSNE1', 'tSNE2', color='in_genera_trimmed',
          template='plotly_white',
           category_orders={'in_genera_trimmed': label_order},
          title='Diversity of compounds with between 20–60 heavy atoms (tSNE by Tanomoto similarity on 1024bit Morgan fingerprints)',
           labels={'in_genera_trimmed': 'Genus'},
           height=800,
          )

In [ ]:
d = (molregno.tSNE1**2 + molregno.tSNE2**2)**0.5

sub = molregno.loc[d.sort_values(ascending=True).index]
sub = sub.loc[(sub.HAC < 60) & (sub.HAC >= 20)].head(20)
Draw.MolsToGridImage(sub.mol.to_list(), legends=sub.in_families_trimmed.to_list(), molsPerRow=5)

In [ ]:
sub = molregno.loc[d.sort_values(ascending=False).index]
sub = sub.loc[(sub.HAC < 60) & (sub.HAC >= 20)].head(50)
Draw.MolsToGridImage(sub.mol.to_list(), legends=sub.in_genera_trimmed.to_list(), molsPerRow=5)

In [ ]:
print(summary.T.sort_values('Kd', ascending=False).to_markdown())

In [ ]:
for activities.genus_id.value_counts()


activities.drop_duplicates(['molregno'])\
          .groupby(['standard_type', 'genus_id'])\
          .size().unstack(fill_value=0)\
          .drop(columns=[-1])\
          .loc[activities.standard_type.value_counts().head(6).index]\
          .rename(columns=taxid_to_name)
summary.style.format("{:,}")